<a href="https://colab.research.google.com/github/valterlucena/recuperacao-informacao/blob/master/vectorial-model/vectorial_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# Introdução

Nesta atividade iremos exercitar algumas instanciações do modelo vetorial.

Primeiramente, vamos importar nossos dados.

In [0]:
DATA_URL = 'https://raw.githubusercontent.com/Benardi/ri_lab_01/master/output/results.csv'
news = pd.read_csv(DATA_URL)

Agora iremos criar nosso índice invertido, para ser utilizado mais adiante. Utilizaremos a função tokenize da biblioteca NLTK associada à uma expressão regular, que considerará como token apenas as sequências de caracteres não-especiais (com exceção do hífen) ou numéricos que não formem stopwords e quem possuam mais que 2 caracteres.
Além disso, refinaremos nosso índice para que o mesmo contenha o *inverse document frequency* (IDF) de cada *posting*.

In [0]:
def isValid(token):
  return not bool(re.search(r'\d', token)) and len(token) > 2

def total_documents():
  return len(news)

def get_tokens(document):
  toker = RegexpTokenizer('''\w+[-']*\w*''')
  stop_words = stopwords.words('portuguese')
  return [token for token in toker.tokenize(document.lower()) if isValid(token) and token not in stop_words]

def build_index(documents):
  index = {}
  n = 0
  for document in documents:
    n += 1
    tokens = get_tokens(document)
    for token in tokens:
      occurrence = tokens.count(token)
      if token not in index:
        index[token] = {}
      if n not in index[token]:
        index[token][n] = occurrence
  return index

index = build_index(news.text)

for posting in index:
  k = len(index[posting])
  idf = round(np.log((total_documents() + 1) / k), 2)
  index[posting]['idf'] = idf

Um vocabulário dos termos presentes nos documentos nos auxiliará em algumas das instanciações que faremos.

In [0]:
vocabulary = index.keys()

# Instanciações



In [0]:
'''
  Calcula a medida de similaridade entre a consulta e um documento
  pela representação binária do modelo vetorial
'''
def binary_representation(query, document):
  terms = query.split()
  doc_tokens = get_tokens(document)
  q = {}
  d = {}
  for term in terms:
    q[term] = 0
    d[term] = 0
    if term in vocabulary:
      q[term] = 1
    if term in doc_tokens:
      d[term] = 1  
  measure = 0
  for term in terms:
    if q[term] != 0 and d[term] != 0:
      measure += q[term] * d[term]  
  return measure
  
'''
  Calcula a medida de similaridade entre a consulta e um documento
  pela representação TF do modelo vetorial
'''
def tf_representation(query, document):
  terms = query.split()
  doc_tokens = get_tokens(document)
  q = {}
  d = {}
  for term in terms:
    q[term] = 0
    d[term] = 0
    if term in vocabulary:
      q[term] = terms.count(term)
    if term in doc_tokens:
      d[term] = doc_tokens.count(term)
  measure = 0
  for term in terms:
    if q[term] != 0 and d[term] != 0:
      measure += q[term] * d[term]
  return measure

'''
  Calcula a medida de similaridade entre a consulta e um documento
  pela representação TF-IDF do modelo vetorial
'''
def tf_idf_representation(query, document):
  terms = query.split()
  doc_tokens = get_tokens(document)
  q = {}
  d = {}
  for term in terms:
    q[term] = 0
    d[term] = 0
    if term in vocabulary:
      q[term] = terms.count(term)
    if term in doc_tokens:
      d[term] = doc_tokens.count(term)
  measure = 0
  for term in terms:
    idf = index[term]['idf']
    if q[term] != 0 and d[term] != 0:
      measure += q[term] * d[term] * idf
  return round(measure, 2)

'''
  Calcula a medida de similaridade entre a consulta e um documento
  pela representação bm25 do modelo vetorial
'''
def bm25_representation(query, document, k):
  terms = query.split()
  doc_tokens = get_tokens(document)
  matched = [term for term in terms if term in doc_tokens]
  measure = 0
  for match in matched:
    cwq = terms.count(match)
    cwd = doc_tokens.count(match)
    m = total_documents()
    dfw = len(index[match].keys()) - 1
    measure += cwq * (((k + 1) * cwd) / (cwd  + k)) * np.log((m + 1) / dfw)
  return round(measure, 2)

# Consultas

As consultas realizadas serão:

* Jair Bolsonaro
* Reforma previdência
* Forças armadas

Para a consulta utilizando o BM25, utilizaremos k = 10.

In [0]:
queries = ['jair bolsonaro', 'reforma previdência', 'forças armadas']

def get_top_5(results):
  return sorted(results, key = lambda x: x[1], reverse=True)[:5]

In [0]:
def get_top_documents(query, documents):
  n = 0
  binary = []
  tf = []
  tf_idf = []
  bm25 = []
  for document in documents:
    binary.append((n,binary_representation(query, document)))
    tf.append((n, tf_representation(query, document)))
    tf_idf.append((n, tf_idf_representation(query, document)))
    bm25.append((n, bm25_representation(query, document, 10)))
    n += 1
  data = {
      'binary': get_top_5(binary),
      'tf': get_top_5(tf),
      'tf_idf': get_top_5(tf_idf),
      'bm25': get_top_5(bm25)
  }
  return data

In [8]:
top_5_q1 = get_top_documents('jair bolsonaro', news.text)
pd.DataFrame(top_5_q1)

,binary,tf,tf_idf,bm25
0,"(0, 2)","(150, 52)","(206, 79.74)","(206, 27.29)"
1,"(1, 2)","(206, 48)","(150, 76.2)","(150, 22.53)"
2,"(24, 2)","(165, 39)","(165, 54.0)","(165, 16.13)"
3,"(85, 2)","(18, 26)","(18, 34.32)","(18, 10.46)"
4,"(125, 2)","(41, 12)","(215, 17.1)","(215, 10.16)"


In [9]:
top_5_q2 = get_top_documents('reforma previdência', news.text)
pd.DataFrame(top_5_q2)

,binary,tf,tf_idf,bm25
0,"(36, 2)","(36, 19)","(36, 43.14)","(36, 23.32)"
1,"(94, 2)","(137, 14)","(137, 31.0)","(137, 20.03)"
2,"(137, 2)","(165, 10)","(165, 22.3)","(165, 16.37)"
3,"(139, 2)","(247, 9)","(247, 19.96)","(247, 15.14)"
4,"(165, 2)","(204, 8)","(204, 17.4)","(204, 13.04)"


In [10]:
top_5_q3 = get_top_documents('forças armadas', news.text)
pd.DataFrame(top_5_q3)

,binary,tf,tf_idf,bm25
0,"(0, 2)","(149, 15)","(149, 33.34)","(149, 21.01)"
1,"(5, 2)","(24, 9)","(24, 19.87)","(24, 15.1)"
2,"(11, 2)","(165, 8)","(165, 17.96)","(165, 14.1)"
3,"(24, 2)","(207, 8)","(207, 17.96)","(207, 14.1)"
4,"(41, 2)","(0, 6)","(0, 13.47)","(0, 11.39)"


O modelo binário, por apenas informar se o termo está presente ou não no documento, não fornece resultados satisfatórios. Observamos que, para todas as consultas, os documentos mais relevantes para os modelos restantes possuem posições parecidas, variando não mais que uma posição de um modelo para outro. Entretando, o modelo utilizando TF-IDF pode ser considerado melhor, pois alcança resultados semelhantes com as outras medidas com um cálculo mais simples que o BM25, por exemplo.

# Jaccard index.

O índice de Jaccard é uma medida utilizada para medir similaridade de conjuntos. Utilizaremos este valor para medir a similaridade, par a par, entre os documentos mais relevantes de acordo com os modelos de instanciação implementados.

In [0]:
def intersection_size(a, b):
  counter = 0
  for el in a:
    if el in b:
      counter += 1
  return counter

def jaccard_index(a, b):
  n_a = len(a)
  n_b = len(b)
  n_ab = intersection_size(a, b)
  exp = n_a + n_b - n_ab
  jaccard = n_ab / exp if exp != 0 else 0
  return jaccard

In [0]:
def get_doc_list(dic):
  result = []
  for values in dic.values():
    result.append([doc for doc,_ in values])
  return result


def get_jaccard_results(dic):
  resultados = get_doc_list(dic)
  matriz = [['Measure', 'Binary', 'TF', 'TF-IDF', 'BM25']]
  titles = matriz
  for i in range(len(resultados)):
    linha = []
    for j in range(len(resultados)):
      jaccard = jaccard_index(resultados[i], resultados[j])
      linha.append(round(jaccard,2))
    matriz.append(linha)
  for i in range(1, len(matriz)):
    matriz[i].insert(0, matriz[0][i])
  return matriz

In [15]:
pd.DataFrame(get_jaccard_results(top_5_q1))

,0,1,2,3,4
0,Measure,Binary,TF,TF-IDF,BM25
1,Binary,1,0,0,0
2,TF,0,1,0.67,0.67
3,TF-IDF,0,0.67,1,1
4,BM25,0,0.67,1,1


In [16]:
pd.DataFrame(get_jaccard_results(top_5_q2))

,0,1,2,3,4
0,Measure,Binary,TF,TF-IDF,BM25
1,Binary,1,0.43,0.43,0.43
2,TF,0.43,1,1,1
3,TF-IDF,0.43,1,1,1
4,BM25,0.43,1,1,1


In [17]:
pd.DataFrame(get_jaccard_results(top_5_q3))

,0,1,2,3,4
0,Measure,Binary,TF,TF-IDF,BM25
1,Binary,1,0.25,0.25,0.25
2,TF,0.25,1,1,1
3,TF-IDF,0.25,1,1,1
4,BM25,0.25,1,1,1


Os resultados do modelo binário, como já falado anteriormente, distoa bastante dos resultados das outras instanciações, o que pode ser verificado pelo seu índice de Jaccard. Já as instanciações restantes, por possuírem resultados semelhantes, possuem um valor alto de similaridade entre si, e, no caso do TF-IDF e BM25, valor máximo, já que retornaram os mesmos resultados.